This script creates a raster of forest that is at least 30% canopy cover and has been continuous forest since 2000.
This script uses data from Hansen et al (2025).

In [2]:
# setting up all our packages and stuff
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import torch
import torch.nn as nn
from osgeo import gdal
import rasterio
import rasterio.plot

#from utils import * 
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

plt.rcParams['savefig.dpi'] = 400
plt.rcParams['font.size'] = 13
plt.rcParams["legend.frameon"] = False

In [3]:
# import the Hansen tree cover and tree cover loss rasters

# Hansen tree cover extent, year 2000
mdd_tc_ds = rasterio.open('/home/jovyan/MLEAEEE4000-DroughtAmazon/tce_mdd_01.tif')
mdd_tc_all = mdd_tc_ds.read(1)

# Hansen tree cover loss, year 2001-2024. If a pixel experiences tree cover loss, its value is the year it experiences the loss.
mdd_tcl_ds = rasterio.open('/home/jovyan/MLEAEEE4000-DroughtAmazon/tcl_mdd_01.tif')
mdd_tcl = mdd_tcl_ds.read(1)

In [25]:
# copy tree cover extent dataset and assign 1000 value to anything below 30% canopy cover to have a way of filtering against it.
mdd_tc_30 = mdd_tc_all.copy()
mdd_tc_30[mdd_tc_30<30]= 1000

In [29]:
# add 1000 to all tcl values, so be able to distinguish against them
mdd_tcl[mdd_tcl>0]=1000

In [30]:
# combine mdd_tcl and mdd_tc_30 into one raster to use as the clipping mask
mdd_tc_tcl = mdd_tc_30 + mdd_tcl

In [34]:
# check min and max calues
print('max:',mdd_tc_tcl.max())
print('min:',mdd_tc_tcl.min())

max: 2000.0
min: 30.0


The resulting raster, mdd_tc_tcl has values ranging from 30 to 2000. Any value above 999 should be discarded from the analysis, for it either is below the 30% canopy threshhold, or it was deforested between 2001-2024, meaning it has not been contiunous forest for the past 20 years.

In [51]:
#filename = '/home/jovyan/MLEAEEE4000-DroughtAmazon/tcl_mdd_01.tif'

def read_geotiff(filename):
    ds = gdal.Open(filename)
    band = ds.GetRasterBand(1)
    arr = band.ReadAsArray()
    return arr, ds

In [52]:
#output = '/home/jovyan/MLEAEEE4000-DroughtAmazon/hansen_datamask.tif'

def write_geotiff(output, arr, in_ds):
    if arr.dtype == np.float32:
        arr_type = gdal.GDT_Float32
    else:
        arr_type = gdal.GDT_Int32

    driver = gdal.GetDriverByName("GTiff")
    out_ds = driver.Create(output, arr.shape[1], arr.shape[0], 1, arr_type)
    out_ds.SetProjection(in_ds.GetProjection())
    out_ds.SetGeoTransform(in_ds.GetGeoTransform())
    band = out_ds.GetRasterBand(1)
    band.WriteArray(arr)
    band.FlushCache()
    band.ComputeStatistics(False)

In [54]:
tc_arr, tc_ds = read_geotiff("/home/jovyan/MLEAEEE4000-DroughtAmazon/tcl_mdd_01.tif")

In [55]:
write_geotiff("/home/jovyan/MLEAEEE4000-DroughtAmazon/hansen_datamask.tif",mdd_tc_tcl, tc_ds)